# Neural Network Matrix Factorization
> Reference: Dziugaite, Gintare & Roy, Daniel. (2015). Neural Network Matrix Factorization. 

### Data organization
We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{f},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We express spatio-temporal dataset as a matrix $Y\in\mathbb{R}^{m\times f}$ with $m$ rows (e.g., locations) and $f$ columns (e.g., discrete time intervals),

$$Y=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{m1} & y_{m2} & \cdots & y_{mf} \\ \end{array} \right]\in\mathbb{R}^{m\times f}.$$

### Matrix factorization
The incomplete matrix $Y$ can be approximated by the product of to much smaller submatrices $\boldsymbol{W}\in\mathbb{R}^{m\times r}$ and $\boldsymbol{X}\in\mathbb{R}^{f\times r}$. $r$ is the rank of submatrices $\boldsymbol{W}$ and $\boldsymbol{X}$. The matrix facotrization(MF) process is as follows:
$$
Y \approx \boldsymbol{W}^\top\boldsymbol{X}
$$

## Neural network matrix factorization
Neural network matrix factorization(NNMF) is a matrix factorization method fully based on neural networks. The inner product of two submatrices in the scheme of MF is replaced by a fully connected neural network. Let's use $w_i \in \mathbb{R}^r, i = 1, 2, ..., m$ and $x_t\in \mathbb{R}^r, t = 1, 2, ..., f$ to denote the lowrank spatial and temporal embeddings respectively and use $f(w_i, x_t)$ to denote the feed forward process of the fully connected neural network. Then the neural network matrix factorization can be described as:
$$ y_{it} \approx f(w_i, x_t) $$

Neural network matrix factorization can be trained by iteratively updating the spatial/temporal embeddings (i.e. $w_i$ and $x_t$) and the weights in the fully connected neuralnetwork. The loss function of the neural network is as follows:
$$
L = \sum_{(i,t)\in\Omega}(y_{it} - f(w_i, x_t))^2 + \lambda\biggl(\sum_{i = 1}^m||w_i||_F^2 + \sum_{t = 1}^f||x_t||_F^2 \biggr)
$$

In [2]:
from __future__ import absolute_import, print_function
import math
import tensorflow as tf
import pandas as pd
import os
import numpy as np

In [3]:
import scipy.io
def minmaxScaler_fitrans(data, scind = 1, scrange_aft = [0,1]):
    maximum = np.max(data)
    minimum = np.min(data)
    if scind == 0:
        scrange_aft = [minimum, maximum]
    scdata = (scrange_aft[1] - scrange_aft[0]) * (data - minimum) / (maximum - minimum) + scrange_aft[0]
    scrange_init = [minimum, maximum]
    return scdata, scrange_init, scrange_aft

def load_data(scind = 1, scrange = [0, 1]):
    tensor = scipy.io.loadmat('../Guangzhou-data-set/tensor.mat')
    tensor = tensor['tensor']
    random_matrix = scipy.io.loadmat('../Guangzhou-data-set/random_matrix.mat')
    random_matrix = random_matrix['random_matrix']
    random_tensor = scipy.io.loadmat('../Guangzhou-data-set/random_tensor.mat')
    random_tensor = random_tensor['random_tensor']

    dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
    missing_rate = 0.2

    # =============================================================================
    ### Random missing (RM) scenario
    ### Set the RM scenario by:
    binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
                                                                       random_tensor.shape[1] 
                                                                       * random_tensor.shape[2]])
    # =============================================================================

    # =============================================================================
    ### Non-random missing (NM) scenario
    ### Set the NM scenario by:
    # binary_tensor = np.zeros(tensor.shape)
    # for i1 in range(tensor.shape[0]):
    #     for i2 in range(tensor.shape[1]):
    #         binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
    # binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] 
    #                                     * binary_tensor.shape[2]])
    # =============================================================================
    sparse_mat = np.multiply(dense_mat, binary_mat)
    scale_mat, scrange_init, scrange_aft = minmaxScaler_fitrans(dense_mat,
                                                                scind = 1,
                                                                scrange_aft = scrange)
    test_pos = np.where((sparse_mat == 0) & (dense_mat != 0))
    train_pos = np.where((sparse_mat != 0) & (dense_mat != 0))
    test_data = pd.DataFrame({
        'road':test_pos[0],
        'time_slot':test_pos[1],
        'speed_target':scale_mat[test_pos]
    })
    trainvalid_data = pd.DataFrame({
        'road':train_pos[0],
        'time_slot':train_pos[1],
        'speed_target':scale_mat[train_pos]
    })
    msk = np.random.rand(len(trainvalid_data)) < 0.9
    train_data = trainvalid_data[msk]
    valid_data = trainvalid_data[~msk]
    return train_data, valid_data, test_data, scrange_init, scrange_aft

In [4]:
def _weight_init_range(n_in, n_out):
    """Calculates range for picking initial weight values from a uniform distribution."""
    range = 4.0 * math.sqrt(6.0) / math.sqrt(n_in + n_out)
    return {
        'minval': -range,
        'maxval': range,
    }

In [5]:
def build_mlp(f_input_layer, hidden_units_per_layer, dprate):
    """Builds a feed-forward NN (MLP) with 3 hidden layers."""
    # Note: tf.contrib.layers could likely be used instead, but total control allows for easier debugging in this case
    # TODO make number of hidden layers a parameter, if needed
    num_f_inputs = f_input_layer.get_shape().as_list()[1]

    # MLP weights picked uniformly from +/- 4*sqrt(6)/sqrt(n_in + n_out)
    mlp_weights = {
        'h1': tf.Variable(tf.random_uniform([num_f_inputs, hidden_units_per_layer],
                                            **_weight_init_range(num_f_inputs, hidden_units_per_layer))),
        'b1': tf.Variable(tf.zeros([hidden_units_per_layer])),
        'h2': tf.Variable(tf.random_uniform([hidden_units_per_layer, hidden_units_per_layer],
                                            **_weight_init_range(hidden_units_per_layer, hidden_units_per_layer))),
        'b2': tf.Variable(tf.zeros([hidden_units_per_layer])),
        'h3': tf.Variable(tf.random_uniform([hidden_units_per_layer, hidden_units_per_layer],
                                            **_weight_init_range(hidden_units_per_layer, hidden_units_per_layer))),
        'b3': tf.Variable(tf.zeros([hidden_units_per_layer])),
        'out': tf.Variable(tf.random_uniform([hidden_units_per_layer, 1],
                                            **_weight_init_range(hidden_units_per_layer, 1))),
        'b_out': tf.Variable(tf.zeros([1])),
    }
    # MLP layers
    mlp_layer_1 = tf.nn.sigmoid(tf.matmul(f_input_layer, mlp_weights['h1']) + mlp_weights['b1'])
    drop_out_1 = tf.nn.dropout(mlp_layer_1, dprate[0])
    mlp_layer_drop_2 = tf.nn.sigmoid(tf.matmul(drop_out_1, mlp_weights['h2']) + mlp_weights['b2'])
    drop_out_2 = tf.nn.dropout(mlp_layer_drop_2, dprate[1])
#     mlp_layer_drop_3 = tf.nn.sigmoid(tf.matmul(drop_out_2, mlp_weights['h3']) + mlp_weights['b3'])
#     drop_out_3 = tf.nn.dropout(mlp_layer_drop_3, dprate[2])
#     out = tf.matmul(drop_out_3, mlp_weights['out']) + mlp_weights['b_out']
    out = tf.matmul(drop_out_2, mlp_weights['out']) + mlp_weights['b_out']
    mlp_layer_2 = tf.nn.sigmoid(tf.matmul(mlp_layer_1, mlp_weights['h2']) + mlp_weights['b2'])
#     mlp_layer_3 = tf.nn.sigmoid(tf.matmul(mlp_layer_2, mlp_weights['h3']) + mlp_weights['b3'])
    pred = tf.matmul(mlp_layer_2, mlp_weights['out']) + mlp_weights['b_out']
    return out, pred, mlp_weights

In [6]:
def chunk_df(df, size):
    """Splits a Pandas dataframe into chunks of size `size`.

    See here: https://stackoverflow.com/a/25701576/1424734
    """
    return (df[pos:pos + size] for pos in range(0, len(df), size))

In [7]:
class _NNMFBase(object):
    def __init__(self, num_roads, num_times, D=10, scrange_init = [0,1], scrange_aft = [0,1], hidden_units_per_layer=100,
                 latent_normal_init_params={'mean': 0.0, 'stddev': 0.1}, dprate = [1, 1, 1], model_filename='model/nnmf.ckpt'):
        self.num_roads = num_roads
        self.num_times = num_times
        self.D = D
        self.hidden_units_per_layer = hidden_units_per_layer
        self.latent_normal_init_params = latent_normal_init_params
        self.model_filename = model_filename
        # Internal counter to keep track of current epoch
        self._epochs = 0
        self.dprate = dprate
        # Input
        self.road_index = tf.placeholder(tf.int32, [None])
        self.time_index = tf.placeholder(tf.int32, [None])
        self.speed_target = tf.placeholder(tf.float32, [None])

        # Call methods to initialize variables and operations (to be implemented by children)
        self._init_vars()
        self._init_ops()

        # RMSE
        self.rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.speed, self.speed_target))))
        self.scrmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.scspeed, self.speed_target))))
    def _init_vars(self):
        raise NotImplementedError

    def _init_ops(self):
        raise NotImplementedError

    def init_sess(self, sess):
        self.sess = sess
        init = tf.initialize_all_variables()
        self.sess.run(init)

    def _train_iteration(self, data, additional_feed=None):
        road_ids = data['road']
        time_ids = data['time_slot']
        speed_target = data['speed_target']

        feed_dict = {self.road_index: road_ids, self.time_index: time_ids, self.speed_target: speed_target}

        if additional_feed:
            feed_dict.update(additional_feed)

        for step in self.optimize_steps:
            self.sess.run(step, feed_dict=feed_dict)

        self._epochs += 1

    def train_iteration(self, data):
        self._train_iteration(data)

    def eval_loss(self, data):
        raise NotImplementedError

    def eval_rmse(self, data):
        road_ids = data['road']
        time_ids = data['time_slot']
        speed_target = data['speed_target']

        feed_dict = {self.road_index: road_ids, self.time_index: time_ids, self.speed_target: speed_target}
        return self.sess.run(self.rmse, feed_dict=feed_dict)
    
    def descale_eval_rmse(self, data, scrange_init, scrange_aft):
        road_ids = data['road']
        time_ids = data['time_slot']
        speed_target = data['speed_target'] * (scrange_init[1] - scrange_init[0]) / (scrange_aft[1] - scrange_aft[0]) + scrange_init[0]
        feed_dict = {self.road_index: road_ids, self.time_index: time_ids, self.speed_target: speed_target}
        return self.sess.run(self.scrmse, feed_dict=feed_dict)
        
    def predict(self, road_id, time_id):
        rating = self.sess.run(self.speed, feed_dict={self.road_index: [road_id], self.time_index: [time_id]})
        return rating[0]

In [8]:
class NNMF(_NNMFBase):
    def __init__(self, *args, **kwargs):
        if 'lam' in kwargs:
            self.lam = float(kwargs['lam'])
            del kwargs['lam']
        else:
            self.lam = 0.01

        super(NNMF, self).__init__(*args, **kwargs)

    def _init_vars(self):
        # Latents
        self.U = tf.Variable(tf.truncated_normal([self.num_roads, self.D], **self.latent_normal_init_params))
#         self.Uprime = tf.Variable(tf.truncated_normal([self.num_roads, self.Dprime], **self.latent_normal_init_params))
        self.V = tf.Variable(tf.truncated_normal([self.num_times, self.D], **self.latent_normal_init_params))
#         self.Vprime = tf.Variable(tf.truncated_normal([self.num_times, self.Dprime], **self.latent_normal_init_params))

        # Lookups
        self.U_lu = tf.nn.embedding_lookup(self.U, self.road_index)
#         self.Uprime_lu = tf.nn.embedding_lookup(self.Uprime, self.road_index)
        self.V_lu = tf.nn.embedding_lookup(self.V, self.time_index)
#         self.Vprime_lu = tf.nn.embedding_lookup(self.Vprime, self.time_index)

        # MLP ("f")
        f_input_layer = tf.concat(axis=1, values=[self.U_lu, self.V_lu])

        _speed, _pred, self.mlp_weights = build_mlp(f_input_layer, self.hidden_units_per_layer, self.dprate)
        self.speed = tf.squeeze(_speed, squeeze_dims=[1])
        self.pred = tf.squeeze(_pred, squeeze_dims=[1])
        self.scspeed = self.pred * (scrange_init[1] - scrange_init[0]) / (scrange_aft[1] - scrange_aft[0]) + scrange_init[0]

    def _init_ops(self):
        # Loss
        reconstruction_loss = tf.reduce_sum(tf.square(tf.subtract(self.speed_target, self.speed)), reduction_indices=[0])
        reg = tf.add_n([tf.reduce_sum(tf.square(self.U), reduction_indices=[0,1]),
                        tf.reduce_sum(tf.square(self.V), reduction_indices=[0,1])])
        self.loss = reconstruction_loss + (self.lam*reg)
        # Optimizer
        self.optimizer = tf.train.AdamOptimizer()
        # Optimize the MLP weights
        f_train_step = self.optimizer.minimize(self.loss, var_list=list(self.mlp_weights.values()))
        # Then optimize the latents
        latent_train_step = self.optimizer.minimize(self.loss, var_list=[self.U, self.V])
        self.optimize_steps = [f_train_step, latent_train_step]
#         self.optimize_steps = [latent_train_step]

    def eval_loss(self, data):
        road_ids = data['road']
        time_ids = data['time_slot']
        speed_target = data['speed_target']
        feed_dict = {self.road_index: road_ids, self.time_index: time_ids, self.speed_target: speed_target}
        return self.sess.run(self.loss, feed_dict=feed_dict)

In [9]:
def train(model, sess, saver, train_data, valid_data, test_data, batch_size, max_epochs, use_early_stop, early_stop_max_epoch, scrange_init, scrange_aft):
    # Print initial values
    batch = train_data.sample(batch_size) if batch_size else train_data
    train_error = model.eval_loss(batch)
    train_rmse = model.descale_eval_rmse(batch, scrange_init, scrange_aft)
    valid_rmse = model.descale_eval_rmse(valid_data, scrange_init, scrange_aft)
    print("[start] Train error: {:3f}, Train RMSE: {:3f}; Valid RMSE: {:3f}.".format(train_error, train_rmse, valid_rmse))

    # Optimize
    prev_valid_rmse = float("Inf")
    early_stop_epochs = 0
    for epoch in range(max_epochs):
        # Run (S)GD
        shuffled_df = train_data.sample(frac=1)
        batches = chunk_df(shuffled_df, batch_size) if batch_size else [train_data]

        for batch_iter, batch in enumerate(batches):
            model.train_iteration(batch)

            # Evaluate
            train_error = model.eval_loss(batch)
            train_rmse = model.descale_eval_rmse(batch, scrange_init, scrange_aft)
            valid_rmse = model.descale_eval_rmse(valid_data, scrange_init, scrange_aft)
#             test_rmse = model.descale_eval_rmse(test_data, scrange_init, scrange_aft)
#             print("[{:d}-{:d}] Train error: {:3f}, Train RMSE: {:3f}, Valid RMSE: {:3f}, test RMSE: {:3f}.".format(epoch, batch_iter, train_error, train_rmse, valid_rmse, test_rmse))
            print("[{:d}-{:d}] Train error: {:3f}, Train RMSE: {:3f}, Valid RMSE: {:3f}.".format(epoch, batch_iter, train_error, train_rmse, valid_rmse))
        # Checkpointing/early stopping
        if use_early_stop:
            early_stop_epochs += 1
            if valid_rmse < prev_valid_rmse:
                prev_valid_rmse = valid_rmse
                early_stop_epochs = 0
                saver.save(sess, model.model_filename)
            elif early_stop_epochs == early_stop_max_epoch:
                print("Early stopping ({} vs. {})...".format(prev_valid_rmse, valid_rmse))
                break
        else:
            saver.save(sess, model.model_filename)

In [10]:
def test(model, sess, saver, test_data, scrange_init, scrange_aft, train_data=None, log=True):
    if train_data is not None:
        train_rmse = model.descale_eval_rmse(train_data, scrange_init, scrange_aft)
        if log:
            print("Final train RMSE: {}".format(train_rmse))

    test_rmse = model.descale_eval_rmse(test_data, scrange_init, scrange_aft)
    if log:
        print("Final test RMSE: {}".format(test_rmse))

    return test_rmse

In [11]:
mode = 'train'  #'train' or 'test'
num_roads = 214
num_times = 8784
batch_size = 5000
max_epochs = 1000
scind = 0
scrange = [0, 10]
use_early_stop = True
early_stop_max_epoch = 20
with tf.Session() as sess:
    # Define computation graph & Initialize
    print("Reading in data")
    train_data, valid_data, test_data, scrange_init, scrange_aft = load_data(scind = scind, scrange = scrange)
    print('Building network & initializing variables')
    model = NNMF(num_roads, num_times, lam = 0.01,D=50, hidden_units_per_layer=100,
                 scrange_init = scrange_init, scrange_aft = scrange_aft, dprate = [0.9, 1])
    model.init_sess(sess)
    saver = tf.train.Saver()
    # Train
    # Process data
    if mode == 'train':
        # Create model directory, if needed
        if not os.path.exists(os.path.dirname(model.model_filename)):
            os.makedirs(os.path.dirname(model.model_filename))
        # Train
        train(model, sess, saver, train_data, valid_data, test_data, batch_size, max_epochs, use_early_stop, early_stop_max_epoch, scrange_init, scrange_aft)
    print('Loading best checkpointed model')
    saver.restore(sess, model.model_filename)
    test(model, sess, saver, test_data, scrange_init, scrange_aft, train_data=train_data)

Reading in data


W1212 10:16:05.758572 22256 deprecation.py:506] From <ipython-input-5-a05b88b24df1>:24: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1212 10:16:05.778560 22256 deprecation.py:506] From C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:180: calling squeeze (from tensorflow.python.ops.array_ops) with squeeze_dims is deprecated and will be removed in a future version.
Instructions for updating:
Use the `axis` argument instead


Building network & initializing variables


W1212 10:16:06.157612 22256 deprecation.py:323] From C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\util\tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


[start] Train error: 123571.164062, Train RMSE: 58.464382; Valid RMSE: 58.668446.
[0-0] Train error: 100742.000000, Train RMSE: 52.609852, Valid RMSE: 53.002308.
[0-1] Train error: 85233.132812, Train RMSE: 47.959026, Valid RMSE: 48.131847.
[0-2] Train error: 70535.710938, Train RMSE: 43.205891, Valid RMSE: 43.334518.
[0-3] Train error: 56139.769531, Train RMSE: 38.342171, Valid RMSE: 38.524681.
[0-4] Train error: 45940.972656, Train RMSE: 33.796810, Valid RMSE: 33.707619.
[0-5] Train error: 33998.464844, Train RMSE: 28.633722, Valid RMSE: 28.937582.
[0-6] Train error: 24802.972656, Train RMSE: 24.096735, Valid RMSE: 24.305124.
[0-7] Train error: 18085.042969, Train RMSE: 19.608406, Valid RMSE: 19.960335.
[0-8] Train error: 13006.851562, Train RMSE: 16.099977, Valid RMSE: 16.145369.
[0-9] Train error: 9966.691406, Train RMSE: 13.442271, Valid RMSE: 13.266174.
[0-10] Train error: 7987.181152, Train RMSE: 11.866330, Valid RMSE: 11.845629.
[0-11] Train error: 7283.197754, Train RMSE: 11.9

...

[94-257] Train error: 457.630157, Train RMSE: 3.207454, Valid RMSE: 4.223015.
[94-258] Train error: 460.468323, Train RMSE: 3.199085, Valid RMSE: 4.223461.
[94-259] Train error: 445.102264, Train RMSE: 3.150853, Valid RMSE: 4.226073.
[94-260] Train error: 437.132690, Train RMSE: 3.161273, Valid RMSE: 4.223422.
[94-261] Train error: 462.508057, Train RMSE: 3.226697, Valid RMSE: 4.222847.
[94-262] Train error: 438.071686, Train RMSE: 3.122663, Valid RMSE: 4.224305.
[94-263] Train error: 482.767639, Train RMSE: 3.316273, Valid RMSE: 4.225441.
[94-264] Train error: 468.407806, Train RMSE: 3.240225, Valid RMSE: 4.227745.
[94-265] Train error: 456.256531, Train RMSE: 3.185517, Valid RMSE: 4.230886.
[94-266] Train error: 461.217987, Train RMSE: 3.220912, Valid RMSE: 4.232740.
[94-267] Train error: 196.069794, Train RMSE: 3.093429, Valid RMSE: 4.233303.
[95-0] Train error: 433.851257, Train RMSE: 3.084099, Valid RMSE: 4.234400.
[95-1] Train error: 433.261932, Train RMSE: 3.079854, Valid RMSE: 

[95-96] Train error: 447.287048, Train RMSE: 3.185217, Valid RMSE: 4.230371.
[95-97] Train error: 476.687622, Train RMSE: 3.217344, Valid RMSE: 4.230441.
[95-98] Train error: 437.260773, Train RMSE: 3.086618, Valid RMSE: 4.229713.
[95-99] Train error: 460.341309, Train RMSE: 3.200054, Valid RMSE: 4.232010.
[95-100] Train error: 444.250854, Train RMSE: 3.127121, Valid RMSE: 4.238158.
[95-101] Train error: 485.587891, Train RMSE: 3.277055, Valid RMSE: 4.232816.
[95-102] Train error: 478.050537, Train RMSE: 3.270809, Valid RMSE: 4.230375.
[95-103] Train error: 457.843262, Train RMSE: 3.185016, Valid RMSE: 4.229425.
[95-104] Train error: 419.579346, Train RMSE: 3.069173, Valid RMSE: 4.231871.
[95-105] Train error: 452.388733, Train RMSE: 3.156762, Valid RMSE: 4.239332.
[95-106] Train error: 415.076752, Train RMSE: 3.001206, Valid RMSE: 4.235569.
[95-107] Train error: 454.232635, Train RMSE: 3.166613, Valid RMSE: 4.227820.
[95-108] Train error: 456.005127, Train RMSE: 3.138598, Valid RMSE: 

[95-203] Train error: 451.649658, Train RMSE: 3.184397, Valid RMSE: 4.227576.
[95-204] Train error: 464.683411, Train RMSE: 3.182629, Valid RMSE: 4.231976.
[95-205] Train error: 476.847473, Train RMSE: 3.277427, Valid RMSE: 4.229652.
[95-206] Train error: 464.121002, Train RMSE: 3.235594, Valid RMSE: 4.225624.
[95-207] Train error: 478.460205, Train RMSE: 3.274384, Valid RMSE: 4.229414.
[95-208] Train error: 455.245300, Train RMSE: 3.135779, Valid RMSE: 4.230915.
[95-209] Train error: 484.903503, Train RMSE: 3.308082, Valid RMSE: 4.239014.
[95-210] Train error: 489.391418, Train RMSE: 3.348967, Valid RMSE: 4.244902.
[95-211] Train error: 464.207703, Train RMSE: 3.248500, Valid RMSE: 4.235115.
[95-212] Train error: 463.437744, Train RMSE: 3.188486, Valid RMSE: 4.232269.
[95-213] Train error: 448.178711, Train RMSE: 3.156435, Valid RMSE: 4.232263.
[95-214] Train error: 442.843140, Train RMSE: 3.136757, Valid RMSE: 4.226708.
[95-215] Train error: 462.486633, Train RMSE: 3.235756, Valid RM

W1212 11:26:22.517071 22256 deprecation.py:323] From C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


[95-267] Train error: 213.030884, Train RMSE: 3.329441, Valid RMSE: 4.227109.
Early stopping (4.208578109741211 vs. 4.227108955383301)...
Loading best checkpointed model
Final train RMSE: 3.3234102725982666
Final test RMSE: 4.2034010887146
